In [111]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import os
import pickle

## Load Data

In [112]:
df = pd.read_csv(r"C:\Users\nene0\Desktop\Projects\kaggle_Used_Car_Regression\cleaned_train_2024-09-18_16-29-24.csv")

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


#### Create_dictionary function

In [113]:
def create_disctionary(data, group, column):
    mode = data.groupby(group)[column].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()
    my_dict = mode.set_index(group)[column].to_dict()
    return my_dict

### Age & Age Bins

In [114]:
df['age'] = datetime.now().year - df['model_year']
df['age_bins'] = pd.cut(df['age'], bins=[0, 2, 5, 10, 15, 20, df['age'].max()], labels=['Fairly New', '3-5 Years', '6-10 Years', '11-15 Years', '16-20 Years', 'Old'])

### Milage per Year

In [115]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,age,age_bins
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17,16-20 Years
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22,Old
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22,Old
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7,6-10 Years
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3,3-5 Years


In [116]:
df['mi_per_year'] = round(df['milage'] / df['age'], 2)

In [117]:
df[df['mi_per_year']==np.inf]

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,age,age_bins,mi_per_year
5566,Lexus,NX 300h Base,2024,16425,Hybrid,2.5L I4 16V MPFI DOHC Hybrid,Automatic CVT,Black,Gray,None reported,Yes,44700,0,NaN,inf
6139,BMW,330 i xDrive,2024,8700,Gasoline,255.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Blue,White,None reported,Yes,33000,0,NaN,inf
7160,BMW,M3 CS,2024,31850,Gasoline,543.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,22550,0,NaN,inf
7223,Ford,Expedition Timberline,2024,23300,Gasoline,440.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Gray,Black,None reported,Yes,69500,0,NaN,inf
7409,BMW,M3 CS,2024,34254,Gasoline,543.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,White,Black,None reported,Yes,63900,0,NaN,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178315,BMW,M3 CS,2024,3415,Gasoline,543.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,A/T,Black,Black,None reported,Yes,179900,0,NaN,inf
180561,BMW,M3 CS,2024,48200,Gasoline,543.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,67000,0,NaN,inf
184526,RAM,1500 Big Horn,2024,18385,Gasoline,395.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,None reported,Yes,57000,0,NaN,inf
185313,INFINITI,QX56 Base,2024,7100,Gasoline,400.0HP 5.6L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes,74900,0,NaN,inf


In [118]:
df['mi_per_year'] = np.where(df['mi_per_year']==np.inf, df['milage'], df['mi_per_year'])

In [119]:
df['mi_per_year'].iloc[5566]

16425.0

### Horsepower & Engine Liter & Cylinder

In [120]:
def extract_horsepower(hp):
    try:
        pattern = r'\d+\.\d+HP'
        match = re.search(pattern, hp)
        hp_string = match.group()
        horsepower = hp_string.replace('.0HP', '')
        horsepower = int(horsepower)
        return horsepower
    except:
        return np.nan
    
def extract_engine_liter(liter):
    try:
        pattern = r'\d+\.\d+L'
        match = re.search(pattern, liter)
        liter_string = match.group()
        liter = liter_string.replace('L', '')
        liter = float(liter)
        return liter
    except:
        return np.nan
    
def extract_cylinder(cyn):
    try:
        pattern = r'\d+ Cylinder'
        match = re.search(pattern, cyn)
        cyn_string = match.group()
        cylinder = cyn_string.replace(' Cylinder', '')
        cylinder = int(cylinder)
        return cylinder
    except:
        return np.nan

In [121]:
# Create a 'horsepower', 'engine_liter', 'cylinder' features using the 'engine'.
df['hp'] = df['engine'].apply(extract_horsepower)
df['engine_liter'] = df['engine'].apply(extract_engine_liter)
df['cylinder'] = df['engine'].apply(extract_cylinder)

In [122]:
hp_median = df['hp'].median()
engine_liter_median = df['engine_liter'].median()
cylinder_median = df['cylinder'].median()

# Function to fill missing values with medians
def fill_missing_values(row):
    row['hp'] = row['hp'] if not pd.isna(row['hp']) else hp_median
    row['engine_liter'] = row['engine_liter'] if not pd.isna(row['engine_liter']) else engine_liter_median
    row['cylinder'] = row['cylinder'] if not pd.isna(row['cylinder']) else cylinder_median
    return row

In [123]:
df = df.apply(fill_missing_values, axis=1)

#### Correcting Horsepower

In [124]:
df[df['hp'] == 1020]

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,age,age_bins,mi_per_year,hp,engine_liter,cylinder
3244,Tesla,Model S Plaid,2021,18000,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,Red,Black,None reported,Yes,55000,3,3-5 Years,6000.00,1020.0,3.5,6.0
13551,Tesla,Model X Plaid,2023,200,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,85000,1,Fairly New,200.00,1020.0,3.5,6.0
18664,Tesla,Model X Plaid,2022,26500,Electric,1020.0HP Electric Motor Electric Fuel System,1-Speed A/T,Blue,Black,None reported,Yes,49900,2,Fairly New,13250.00,1020.0,3.5,6.0
24009,Tesla,Model X Plaid,2021,44000,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,Red,Black,None reported,Yes,54000,3,3-5 Years,14666.67,1020.0,3.5,6.0
36210,Tesla,Model S Plaid,2021,105000,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,Red,Black,At least 1 accident or damage reported,Yes,20500,3,3-5 Years,35000.00,1020.0,3.5,6.0
37420,Tesla,Model S Plaid,2022,4820,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,White,White,None reported,Yes,110000,2,Fairly New,2410.00,1020.0,3.5,6.0
39613,Tesla,Model X Plaid,2022,20000,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,White,White,None reported,Yes,115000,2,Fairly New,10000.00,1020.0,3.5,6.0
46303,Tesla,Model X Plaid,2022,15000,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,Blue,Black,At least 1 accident or damage reported,Yes,54900,2,Fairly New,7500.00,1020.0,3.5,6.0
51400,Tesla,Model X Plaid,2023,20100,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,Red,White,None reported,Yes,49700,1,Fairly New,20100.00,1020.0,3.5,6.0
51728,Tesla,Model X Plaid,2021,20000,Electric,1020.0HP Electric Motor Electric Fuel System,A/T,White,White,None reported,Yes,56900,3,3-5 Years,6666.67,1020.0,3.5,6.0


In [125]:
hp_dict = create_disctionary(df, 'model', 'hp')
pickle.dump(hp_dict, open("hp_dict.pickle", "wb"))

In [126]:
hp_dict.get('Mustang Mach-E GT')

480.0

In [127]:
df['hp'] = df.apply(lambda row: hp_dict[row['model']] if row['model'] in hp_dict and row['hp'] != hp_dict[row['model']] else row['hp'], axis=1)

In [128]:
df[df['hp'] == 1020]['brand'].unique()

array(['Tesla'], dtype=object)

### Horsepower per Liter

In [129]:
# Horsepower per liter: a measure of an engine's efficiency

df['engine_efficiency'] = round(df['hp'] / df['engine_liter'], 2)

In [130]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,age,age_bins,mi_per_year,hp,engine_liter,cylinder,engine_efficiency
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17,16-20 Years,12529.41,172.0,1.6,4.0,107.50
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22,Old,6511.36,252.0,3.9,8.0,64.62
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22,Old,6215.05,360.0,5.3,8.0,67.92
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7,6-10 Years,2785.71,420.0,5.0,8.0,84.00
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3,3-5 Years,2462.67,208.0,2.0,4.0,104.00


### Brand Popularity

In [131]:
brand_counts = df['brand'].value_counts()
df['brand_count'] = df['brand'].map(brand_counts)
df['brand_rank'] = df['brand_count'].rank(method='dense', ascending=False)

In [146]:
pickle.dump(brand_counts, open("brand_counts.pickle", "wb"))

### Transmission & Num Gear

In [132]:
def extract_gear(gear):
    try:
        pattern = r'^\d+'
        match = re.search(pattern, gear)
        num_gear = match.group()
        return num_gear
    except:
        return np.nan

In [133]:
df['num_gear'] = df['transmission'].apply(extract_gear)

In [134]:
df[df['transmission']=='Single-Speed Fixed Gear']

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,age,age_bins,mi_per_year,hp,engine_liter,cylinder,engine_efficiency,brand_count,brand_rank,num_gear
4516,Bentley,Bentayga Speed,2020,22001,Gasoline,6.0L W12 48V PDI DOHC Twin Turbo,Single-Speed Fixed Gear,Beluga Black,Hotspur,None reported,...,4,3-5 Years,5500.25,328.0,6.0,6.0,54.67,1157,27.0,NaN
29492,Porsche,Taycan,2020,2076,Electric,Electric,Single-Speed Fixed Gear,Gray,Beige,None reported,...,4,3-5 Years,519.00,328.0,3.5,6.0,93.71,10554,6.0,NaN
37890,Porsche,Taycan Turbo,2022,3158,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Gray,Black,None reported,...,2,Fairly New,1579.00,328.0,3.5,6.0,93.71,10554,6.0,NaN
39075,Porsche,Taycan Turbo,2020,2076,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Gray,Unknown,None reported,...,4,3-5 Years,519.00,328.0,3.5,6.0,93.71,10554,6.0,NaN
40380,Porsche,Taycan 4S,2023,11089,Gasoline,Dual AC Electric Motors,Single-Speed Fixed Gear,Black,Black,None reported,...,1,Fairly New,11089.00,328.0,3.5,6.0,93.71,10554,6.0,NaN
66152,Porsche,Taycan Turbo,2021,11078,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Black,Beige,None reported,...,3,3-5 Years,3692.67,328.0,3.5,6.0,93.71,10554,6.0,NaN
87772,Porsche,Taycan Base,2020,2116,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Silver,Black,None reported,...,4,3-5 Years,529.00,375.0,3.5,6.0,107.14,10554,6.0,NaN
96793,Porsche,Taycan Turbo,2020,21028,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Black,Beige,None reported,...,4,3-5 Years,5257.00,328.0,3.5,6.0,93.71,10554,6.0,NaN
102902,Porsche,Taycan Turbo,2022,3574,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Gentian Blue Metallic,Black,None reported,...,2,Fairly New,1787.00,328.0,3.5,6.0,93.71,10554,6.0,NaN
108484,Porsche,Taycan Turbo,2022,2076,Electric,Dual AC Electric Motors,Single-Speed Fixed Gear,Black,Black,None reported,...,2,Fairly New,1038.00,328.0,3.5,6.0,93.71,10554,6.0,NaN


In [135]:
df.loc[((df['transmission']=='Single-Speed Fixed Gear')&(df['num_gear'].isna())), 'num_gear'] = 1

In [136]:
def remove_speed(trans):
    pattern = r'\d+-[Ss][Pp][Ee][Ee][Dd]\s*'
    return re.sub(pattern, '', trans).strip()

df['transmission'] = df['transmission'].apply(remove_speed)

In [137]:
df[df['transmission']=='SCHEDULED FOR OR IN PRODUCTION']

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,age,age_bins,mi_per_year,hp,engine_liter,cylinder,engine_efficiency,brand_count,brand_rank,num_gear
15127,BMW,M440 i xDrive,2021,61145,Hybrid,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Alpine White,Black,None reported,...,3,3-5 Years,20381.67,382.0,3.5,6.0,109.14,17021,3.0,NaN
24462,Audi,Q5 2.0T Premium Plus,2019,36486,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,SCHEDULED FOR OR IN PRODUCTION,Gray,Red,None reported,...,5,3-5 Years,7297.20,220.0,2.0,4.0,110.00,10898,5.0,NaN
74912,BMW,M240 i,2021,13177,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Brilliant Silver Metallic,Black,None reported,...,3,3-5 Years,4392.33,335.0,3.5,6.0,95.71,17021,3.0,NaN
76121,BMW,M3 CS,2022,26609,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Mythos Black Metallic,White,None reported,...,2,Fairly New,13304.50,453.0,3.5,6.0,129.43,17021,3.0,NaN
115203,BMW,X6 xDrive40i,2020,28567,Gasoline,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Carbon Black Metallic,Black,At least 1 accident or damage reported,...,4,3-5 Years,7141.75,335.0,3.5,6.0,95.71,17021,3.0,NaN
136712,BMW,530 i xDrive,2019,48156,Hybrid,3.0 Liter Turbo,SCHEDULED FOR OR IN PRODUCTION,Glacier Silver Metallic,Canberra Beige/Black,None reported,...,5,3-5 Years,9631.20,328.0,3.5,6.0,93.71,17021,3.0,NaN


In [138]:
# Function to replace 'A/T' with 'Automatic' and 'M/T' with 'Manual'
def replace_transmission(trans):
    if 'A/T' in trans:
        return trans.replace('A/T', 'Automatic')
    elif 'M/T' in trans:
        return trans.replace('M/T', 'Manual')
    elif 'Variable' in trans:
        return trans.replace('Variable', 'CVT')
    else:
        return trans

# Apply the function to the 'transmission' column
df['transmission'] = df['transmission'].apply(replace_transmission)

In [139]:
correct_transmission = {
    'CVT Transmission': 'CVT',
    'Automatic CVT': 'CVT',
    'Electronically Controlled Automatic with O':'Automatic with Overdrive',
    'F': np.nan,
    'CVT-F':'CVT',
    '2': np.nan,
    '6 Speed At/Mt': 'Automated Manual',
    '': np.nan,
    'AT':'Automatic',
    'SCHEDULED FOR OR IN PRODUCTION': np.nan,
    '6 Speed Mt': 'Manual'}

In [140]:
df['transmission'] = df['transmission'].replace(correct_transmission)

In [141]:
df['transmission'].fillna(df['transmission'].mode()[0], inplace=True)
df['num_gear'].fillna(df['num_gear'].mode()[0], inplace=True)

### Drop features

In [142]:
df.drop(columns=['model_year', 'engine', 'brand_count'], inplace=True)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188373 entries, 0 to 188372
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   brand              188373 non-null  object 
 1   model              188373 non-null  object 
 2   milage             188373 non-null  int64  
 3   fuel_type          188373 non-null  object 
 4   transmission       188373 non-null  object 
 5   ext_col            188373 non-null  object 
 6   int_col            188373 non-null  object 
 7   accident           188373 non-null  object 
 8   clean_title        188373 non-null  object 
 9   price              188373 non-null  int64  
 10  age                188373 non-null  int64  
 11  age_bins           188211 non-null  object 
 12  mi_per_year        188373 non-null  float64
 13  hp                 188373 non-null  float64
 14  engine_liter       188373 non-null  float64
 15  cylinder           188373 non-null  float64
 16  en

In [144]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
folder_path = r'C:\Users\nene0\Desktop\Projects\kaggle_Used_Car_Regression'
file_path = os.path.join(folder_path, f'featured_train_{timestamp}.csv')
df.to_csv(file_path, index=False)